In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import models, layers
import matplotlib.pyplot as plt
import numpy as np


In [ ]:
IMAGE_SIZE=256
BATCH_SIZE=32  # numher of samples
CHANNELS=3
EPOCHS=100

In [ ]:
dataset= tf.keras.preprocessing.image_dataset_from_directory(
"D:/Test_RC",
   shuffle=True,
    image_size=(IMAGE_SIZE,IMAGE_SIZE),
    batch_size=BATCH_SIZE
)

Found 1335 files belonging to 3 classes.


In [ ]:
class_names=dataset.class_names
class_names

['BrownSpot', 'Healthy', 'LeafBlast']

In [ ]:
len(dataset)

42

In [ ]:
for image_batch, label_batch in dataset.take(1):
    print(image_batch.shape)
    print(label_batch.numpy())

(32, 256, 256, 3)
[1 1 1 1 1 1 1 1 0 0 0 0 0 2 1 1 1 1 1 0 1 0 0 0 1 1 0 1 1 1 0 1]


In [ ]:
for image_batch, label_batch in dataset.take(1):
    print(image_batch[0])

tf.Tensor(
[[[249.45996  252.97754  253.31738 ]
  [247.51172  252.38184  254.68262 ]
  [248.00684  252.83105  254.56934 ]
  ...
  [218.01855  218.01758  244.01172 ]
  [220.14648  219.38965  245.53613 ]
  [217.9961   219.1836   244.58984 ]]

 [[251.76465  254.87012  254.15625 ]
  [250.61035  254.95215  254.4873  ]
  [250.30273  254.24316  254.77441 ]
  ...
  [218.03125  216.03125  237.28809 ]
  [222.95215  220.95215  243.58789 ]
  [223.08887  221.08887  243.9834  ]]

 [[252.98242  255.       254.98145 ]
  [251.       255.       255.      ]
  [248.21875  253.0625   254.93945 ]
  ...
  [222.75098  220.75098  242.81055 ]
  [222.55566  220.55566  242.27148 ]
  [222.80566  220.80566  243.65527 ]]

 ...

 [[254.       254.60645  252.64355 ]
  [252.19434  252.16992  252.29199 ]
  [254.03125  252.09375  253.09082 ]
  ...
  [111.356445 118.325195  76.48047 ]
  [ 79.67676   90.694336  22.777344]
  [ 91.163086 107.549805  17.326172]]

 [[252.       252.4375   253.78125 ]
  [253.8291   254.1709   2

In [ ]:
for image_batch, label_batch in dataset.take(1):
    print(image_batch[0].numpy)

<bound method _EagerTensorBase.numpy of <tf.Tensor: shape=(256, 256, 3), dtype=float32, numpy=
array([[[223.36328, 220.36328, 213.     ],
        [226.42049, 223.42049, 215.36328],
        [224.     , 221.     , 213.51982],
        ...,
        [238.19832, 241.19832, 232.04807],
        [239.99281, 244.18164, 236.71938],
        [244.33029, 245.33029, 239.33029]],

       [[223.51947, 220.51947, 212.62758],
        [224.66217, 221.66217, 212.66217],
        [224.36328, 221.36328, 213.53668],
        ...,
        [240.4949 , 243.4949 , 236.4949 ],
        [239.75201, 242.75201, 235.75201],
        [240.28258, 241.28258, 235.28258]],

       [[223.74324, 220.74324, 213.74324],
        [228.4094 , 225.4094 , 217.33604],
        [222.92505, 219.92505, 214.57472],
        ...,
        [240.63281, 243.63281, 235.96585],
        [235.86642, 238.86642, 231.86642],
        [239.74324, 242.74324, 233.5969 ]],

       ...,

       [[243.1836 , 243.13358, 233.48018],
        [247.33171, 244.45703,

In [ ]:
for image_batch, label_batch in dataset.take(1):
    print(image_batch[0].shape)

(256, 256, 3)


In [ ]:
train_size=0.8
len(dataset) * train_size

33.6

In [ ]:
train_ds=dataset.take(54)
len(train_ds)

42

In [ ]:
test_ds=dataset.skip(54)
len(test_ds)

0

In [ ]:
val_size=0.1
len(dataset)*val_size

4.2

In [ ]:
val_ds=test_ds.take(6)
len(val_ds)

0

In [ ]:
test_ds=test_ds.skip(6)
len(test_ds)

0

In [ ]:
def get_dataset_partitions_tf(ds,train_split=0.8, val_split=0.1, test_split=0.1, shuffle=True, shuffle_size=10000):
    
    ds_size=len(ds)
    
    if shuffle:
        ds=ds.shuffle(shuffle_size, seed=12)
        
    train_size=int(train_split*ds_size)
    val_size=int(val_split*ds_size)
    
    train_ds  = ds.take(train_size)
    val_ds    = ds.skip(train_size).take(val_size)
    test_ds    = ds.skip(train_size).skip(val_size)
    
    
    
    return train_ds, val_ds, test_ds


In [ ]:
train_ds, val_ds, test_ds = get_dataset_partitions_tf(dataset)

In [ ]:
len(train_ds)


33

In [ ]:
len(val_ds)


4

In [ ]:
len(test_ds)

5

In [ ]:
train_ds= train_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
val_ds= val_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
test_ds= test_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)

In [ ]:
resize_and_rescale = tf.keras.Sequential([
    
    layers.experimental.preprocessing.Resizing(IMAGE_SIZE, IMAGE_SIZE),  
    layers.experimental.preprocessing.Rescaling(1.0/255)   
    
])

In [ ]:
data_augmentation =  tf.keras.Sequential([
    layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
    layers.experimental.preprocessing.RandomRotation(0.2),
     
    
])

In [ ]:
input_shape=(BATCH_SIZE, IMAGE_SIZE, IMAGE_SIZE, CHANNELS)
n_classes = 3

model=models.Sequential([
    resize_and_rescale,
    data_augmentation,
    # l1
    layers.Conv2D(32, (3,3), activation = 'relu', input_shape=input_shape),
    layers.MaxPooling2D((2,2)),  # --> downsample (reduces) the vectors
    # l2
    layers.Conv2D(64, kernel_size=(3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),
    # l3
    layers.Conv2D(64, kernel_size=(3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),
    # l4
    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),
    # l5
    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),
    # l6
    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),

    layers.Flatten(),  # mulitdimention val to single dimention

    # Dense 1
    layers.Dense(64, activation='relu'),
    # Dense 2
    layers.Dense(n_classes, activation='softmax'), # vetors to probabilities 
    
    
])

model.build(input_shape=input_shape)

In [ ]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential (Sequential)     (32, 256, 256, 3)         0         
                                                                 
 sequential_1 (Sequential)   (32, 256, 256, 3)         0         
                                                                 
 conv2d (Conv2D)             (32, 254, 254, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2D  (32, 127, 127, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (32, 125, 125, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (32, 62, 62, 64)         0         
 2D)                                                  

In [ ]:
model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)

In [ ]:
history = model.fit(
    train_ds,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    verbose=1,
    validation_data=val_ds
)

Epoch 1/100
33/33 [==============================] - ETA: 0s - loss: 1.0810 - accuracy: 0.4223

In [ ]:
history.history['accuracy']

[0.42518940567970276,
 0.438446968793869,
 0.44034090638160706,
 0.42424243688583374,
 0.4412878751754761,
 0.5340909361839294,
 0.560606062412262,
 0.5909090638160706,
 0.6013257503509521,
 0.5928030014038086,
 0.6164772510528564,
 0.6155303120613098,
 0.6174242496490479,
 0.6278409361839294,
 0.6344696879386902,
 0.6628788113594055,
 0.6780303120613098,
 0.6931818127632141,
 0.71875,
 0.7395833134651184,
 0.7291666865348816,
 0.7547348737716675,
 0.7604166865348816,
 0.7689393758773804,
 0.7651515007019043,
 0.779356062412262,
 0.779356062412262,
 0.7954545617103577,
 0.8096590638160706,
 0.8030303120613098,
 0.7897727489471436,
 0.8229166865348816,
 0.7992424368858337,
 0.7992424368858337,
 0.810606062412262,
 0.8153409361839294,
 0.8248106241226196,
 0.8295454382896423,
 0.8333333134651184,
 0.8276515007019043,
 0.8390151262283325,
 0.8428030014038086,
 0.8238636255264282,
 0.8494318127632141,
 0.8323863744735718,
 0.8541666865348816,
 0.8513257503509521,
 0.8494318127632141,
 0.86

In [ ]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential (Sequential)     (None, 256, 256, 3)       0         
                                                                 
 sequential_1 (Sequential)   (None, 256, 256, 3)       0         
                                                                 
 conv2d (Conv2D)             (None, 254, 254, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 127, 127, 32)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 125, 125, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 62, 62, 64)       0         
 2D)                                                  

In [ ]:
savedModel=load_model('D:/saved_models4/RC_Model_4.h5')
savedModel.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential (Sequential)     (None, 256, 256, 3)       0         
                                                                 
 sequential_1 (Sequential)   (None, 256, 256, 3)       0         
                                                                 
 conv2d (Conv2D)             (None, 254, 254, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 127, 127, 32)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 125, 125, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 62, 62, 64)       0         
 2D)                                                  

In [ ]:
# Training the saved model again ...........
savedModel.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)

In [ ]:
history = savedModel.fit(
    train_ds,
    epochs=200,
    batch_size=BATCH_SIZE,
    verbose=1,
    validation_data=val_ds
)

Epoch 1/200
33/33 [==============================] - 55s 1s/step - loss: 0.1442 - accuracy: 0.9527 - val_loss: 0.1017 - val_accuracy: 0.9766
Epoch 2/200
33/33 [==============================] - 28s 834ms/step - loss: 0.0650 - accuracy: 0.9792 - val_loss: 0.0859 - val_accuracy: 0.9844
Epoch 3/200
33/33 [==============================] - 27s 819ms/step - loss: 0.0654 - accuracy: 0.9763 - val_loss: 0.1037 - val_accuracy: 0.9688
Epoch 4/200
33/33 [==============================] - 27s 813ms/step - loss: 0.0733 - accuracy: 0.9754 - val_loss: 0.0809 - val_accuracy: 0.9922
Epoch 5/200
33/33 [==============================] - 28s 846ms/step - loss: 0.0615 - accuracy: 0.9782 - val_loss: 0.0438 - val_accuracy: 0.9844
Epoch 6/200
33/33 [==============================] - 30s 908ms/step - loss: 0.0427 - accuracy: 0.9830 - val_loss: 0.0575 - val_accuracy: 0.9844
Epoch 7/200
33/33 [==============================] - 27s 826ms/step - loss: 0.0560 - accuracy: 0.9848 - val_loss: 0.1371 - val_accuracy: 0.

In [ ]:
savedModel.save('D:/saved_models5/RC_Model_5.h5')
print('Model Saved!')

Model Saved!
